In [ ]:
from pathlib import Path
import pandas as pd
from matplotlib import pylab as plt
from pymongo import MongoClient

# Load data
Step 0 is pulling events with the Sentry API

In [ ]:
db = MongoClient().fmriprep_stats
all_started_events_df = pd.DataFrame(list(db.started.find()))

In [ ]:
set(all_started_events_df["environment_version"].values)

## Filtering dataframes

In [ ]:
unique_started = all_started_events_df.drop_duplicates(subset=['run_uuid'])
unique_success = all_successful_events_df.drop_duplicates(subset=['tags.run_uuid'])
unique_started_success = unique_success.loc[unique_success['run_uuid'].isin(unique_started['tags.run_uuid'])]

versions = sorted(unique_started["environment_version"].unique())[-12:]

In [ ]:
    # all_events_df.dateCreated = pd.to_datetime(all_events_df.dateCreated)

    # all_events_df["date_minus_time"] = all_events_df["dateCreated"].apply(
    #     lambda df: datetime.datetime(year=df.year, month=df.month, day=df.day)
    # )
    # all_events_df["date_minus_time"] = all_events_df[
    #     "date_minus_time"
    # ] - pd.to_timedelta(7, unit="d")
    # all_events_df.set_index(all_events_df["date_minus_time"], inplace=True)

In [ ]:
versions

In [ ]:
group_success = unique_success.groupby([unique_success['date_minus_time'].dt.year, unique_success['date_minus_time'].dt.week])['id'].count()
grouped_started = unique_started.groupby([unique_started['date_minus_time'].dt.year, unique_started['date_minus_time'].dt.week])['id'].count()
grouped_started_success = unique_started_success.groupby([unique_started_success['date_minus_time'].dt.year, unique_started_success['date_minus_time'].dt.week])['id'].count()

In [ ]:
versions_success = {}
versions_started = {}

for version in versions:
    ver_suc = unique_started_success[unique_started_success['tags.release'] == version]
    ver_sta = unique_started[unique_started['tags.release'] == version]
    
    versions_success[version] = ver_suc.groupby([ver_suc['date_minus_time'].dt.year, ver_suc['date_minus_time'].dt.week])['id'].count()
    versions_started[version] = ver_sta.groupby([ver_sta['date_minus_time'].dt.year, ver_sta['date_minus_time'].dt.week])['id'].count()

versions_success = pd.DataFrame(versions_success)
versions_started = pd.DataFrame(versions_started)


## Successful runs per week

In [ ]:
group_success.index

In [ ]:
sns.set_style("whitegrid")
sns.set_context("poster")
fig,ax = plt.subplots(figsize=(10, 6))
indexes = [f"{w}" for _, w in group_success.index[:-1]]
ax.bar(indexes, group_success.values[:-1], width=0.8, label='Success')
# ax.bar(indexes + 0.2, grouped_started_success.values[:-1], width=0.4,
#        label='Started+Success')

plt.ylabel("Number of successful runs")
plt.xlabel("Week of year (2020-2021)")
plt.title("Successful fMRIPrep runs per week (all versions)")
sns.despine(left=True)
plt.legend(bbox_to_anchor=(1, 1), title='Events');
plt.savefig('20221206_successful_weekly_abs.png', dpi=300, bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(figsize=(10, 6))
indexes = [f"{w}" for _, w in grouped_started.index[:-1]]
ax.bar(indexes, grouped_started.values[:-1], width=0.8, label='Success')
# ax.bar(indexes + 0.2, grouped_started_success.values[:-1], width=0.4,
#        label='Started+Success')

plt.ylabel("Number of executions")
plt.xlabel("Week of year (2020)")
plt.title("Initiated fMRIPrep runs per week (all versions)")
sns.despine(left=True)
plt.legend(bbox_to_anchor=(1, 1), title='Events');
plt.savefig('20221206_all_weekly_abs.png', dpi=300, bbox_inches='tight')

In [ ]:
sns.set_style("whitegrid")
fig,ax = plt.subplots(figsize=(15, 6))
ax.bar(indexes, (grouped_started_success.values[:-1] / grouped_started.values[:-1]) * 100,
       width=0.9, label='Success')

plt.ylabel("Percentage of successful runs")
plt.xlabel("Week of year (2020-2021)")
plt.title("Successful fMRIPrep runs per week (all versions)")
ax.set_ylim((0,100))
sns.despine(left=True)
plt.legend(bbox_to_anchor=(1, 1), title='Events');
plt.savefig('20221206_successful_weekly_rel.png', dpi=300, bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(figsize=(16,6))
ax = versions_success.plot.bar(stacked=True, ax=ax)

sns.despine(left=True)
plt.ylabel("Number of successful runs, broken by version")
plt.xlabel("Week of year")
plt.title("fMRIPrep successful runs per week")
plt.legend(bbox_to_anchor=(1, 1));
plt.savefig('20221206_version_successful_weekly_abs.png', dpi=300, bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(figsize=(16,6))
ax = ((versions_success.T / versions_started.sum(axis=1)).T * 100).plot.bar(stacked=True, ax=ax)

sns.despine(left=True)
ax.set_ylim((0,100))
plt.ylabel("Percentage of total successful runs, broken by version")
plt.xlabel("Week of year")
plt.title("fMRIPrep successful runs per week")
plt.legend(bbox_to_anchor=(1, 1));
plt.savefig('20221206_version_successful_weekly_rel_total.png', dpi=300, bbox_inches='tight')

In [ ]:
fig,ax = plt.subplots(figsize=(16,6))
ax = ((versions_success.sum(axis=0) / versions_started.sum(axis=0)) * 100).plot.bar(ax=ax, width=1.0)

sns.despine(left=True)
ax.set_ylim((0,100))

plt.ylabel("Percentage of total successful runs, broken by version")
plt.xlabel("Week of year")
plt.title("fMRIPrep successful runs per week")
plt.legend(bbox_to_anchor=(1, 1));
plt.savefig('20221206_version_successful_weekly_rel_broken.png', dpi=300, bbox_inches='tight')

In [ ]:
versions_success.sum(axis=0)

In [ ]:
versions_started.sum(axis=0)